In [8]:
import tensorflow as tf
import numpy as np

fd = open('train-images.idx3-ubyte')
train_images = np.fromfile(file=fd,dtype=np.uint8)
fd = open('train-labels.idx1-ubyte')
train_labels = np.fromfile(file=fd,dtype=np.uint8)
fd = open('t10k-images.idx3-ubyte')
test_images = np.fromfile(file=fd,dtype=np.uint8)
fd = open('t10k-labels.idx1-ubyte')
test_labels = np.fromfile(file=fd,dtype=np.uint8)

train_images = (train_images[16:]/127-1).reshape([-1, 784]).astype(np.float)
train_labels = train_labels[8:].reshape([-1,]).astype(np.int)
test_images = (test_images[16:]/127-1).reshape([-1, 784]).astype(np.float)
test_labels = test_labels[8:].reshape([-1]).astype(np.int)

X = tf.placeholder(shape=[None, 784], dtype=tf.float32)
Y = tf.placeholder(shape=[None], dtype=tf.uint8)
W = tf.Variable(tf.zeros([784, 10]))
B = tf.Variable(tf.zeros([10]))
Y_one = tf.one_hot(Y, 10, axis=1)

H = tf.matmul(X, W) + B
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y_one, logits=H))

optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
train_op = optimizer.minimize(loss)

# acc
test_val = tf.nn.softmax(H)
Accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(test_val, 1), tf.argmax(Y_one, 1)), dtype=tf.float32))

#saver
saver = tf.train.Saver()

with tf.Session() as sess:
	sess.run(tf.global_variables_initializer())

	epoch = 10
	for i in range (epoch):
		print("epoch :", i)
		batch_size = 10000
		batch_count = 60000 // batch_size
		loss_total = 0
		for i in range (batch_count):
			batch_index = i * batch_size
			img = train_images[batch_index:batch_index+batch_size]
			label = train_labels[batch_index:batch_index+batch_size]
			# one = np.zeros((batch_index+batch_size,10))
			# one[np.arange(9), label] = 1
			# label = tf.one_hot(train_labels[i:batch_index+batch_size], 10, axis=1)
			_, loss_value = sess.run([train_op, loss], feed_dict={X:img, Y:label})
		loss_total += loss_value
		print("=> loss : ", loss_total)
			# print("batch ",i,"=> loss : ", loss_total)
		saver.save(sess, './weight_image/my-model', global_step=i)
	# acc
	test_img = test_images[:]
	test_label = test_labels[:]
	acc = sess.run([Accuracy], feed_dict={X:test_img, Y:test_label})
	print("Accuracy : ", acc)
	
	img = sess.run(W)

epoch : 0
=> loss :  1.86807203293
epoch : 1
=> loss :  1.44663047791
epoch : 2
=> loss :  1.14996469021
epoch : 3
=> loss :  0.95136153698
epoch : 4
=> loss :  0.81681150198
epoch : 5
=> loss :  0.723884046078
epoch : 6
=> loss :  0.657644987106
epoch : 7
=> loss :  0.608035326004
epoch : 8
=> loss :  0.570468068123
epoch : 9
=> loss :  0.540292978287
Accuracy :  [0.86900008]
